In [159]:
import pandas as pd

data = pd.read_csv(
    'hasil_eda.csv'
    , header=0
)

# 1. Clean Data

## a. Duplikat

In [160]:
# Menemukan duplikat berdasarkan kolom 'teks'
duplikat = data[
    data.duplicated(
        subset=['teks']
        , keep=False
    )
]
# Menampilkan hasil, untuk melihat data duplikat
duplikat.to_csv(
    'duplikat.csv'
    , index=False
)

In [161]:
# Menghapus duplikat berdasarkan kolom 'teks', menyisakan satu saja
data.drop_duplicates(
    subset=['teks']
    , keep='first' # Menyisakan salah satu teks dari duplikat
    , inplace=True # Perubahan langsung diterapkan ke data
)

## b. Outlier

In [162]:
data.drop(
    data[data["is_outlier"] == True].index
    , inplace=True
)
data.to_csv(
    'non_outlier.csv'
    , index=False
)

# 2.Praproses Data

In [163]:
import nltk
import re

In [164]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## c. normalisasi

In [165]:
#Sumber: Medium
def normalize_teks(teks):
    teks = teks.lower()              # huruf kecil semua
    teks = re.sub(r'[^a-zA-Z\s]', '', teks) # hapus semua karakter selain huruf dan spasi
    teks = re.sub(r'\s+', ' ', teks).strip()  # hapus spasi ganda
    return teks

# Terapkan normalisasi pada kolom 'teks'
data['teks_normal'] = data['teks'].apply(normalize_teks)

In [166]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## d. teks stopword

In [167]:
import string
# Stopword Removal
def remove_stopwords(teks_normal):
    tokens = word_tokenize(teks_normal)
    filtered = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(filtered)

data['teks_stopword'] = data['teks_normal'].apply(remove_stopwords)

## e. tokenisasi

In [168]:
# Tokenisasi
def tokenize(teks_stopword):
    return word_tokenize(teks_stopword)

data['teks_tokenisasi'] = data['teks_stopword'].apply(tokenize)

In [169]:
data.to_csv(
    'teks_tokenisasi.csv'
    , index=False
)

## f. Panjang Teks

In [170]:
# Menghitung panjang teks dalam karakter baru
data['panjang_teks'] = data['teks_stopword'].str.len()

In [171]:
# Hitung rata-rata panjang teks untuk AI dan non-AI
rata2_ai = data[data['label'] == 1]['panjang_teks'].mean()
rata2_nonai = data[data['label'] == 0]['panjang_teks'].mean()

# Fungsi untuk menghitung rasio_panjang_teks
def hitung_rasio_panjang(teks_panjang):
    jarak_ke_ai = abs(teks_panjang - rata2_ai)
    jarak_ke_nonai = abs(teks_panjang - rata2_nonai)
    return 1 if jarak_ke_ai < jarak_ke_nonai else 0

# Terapkan ke seluruh data
data['rasio_panjang_teks'] = data['panjang_teks'].apply(hitung_rasio_panjang)


In [172]:
rata2_ai, rata2_nonai

(234.41333333333333, 106.9693094629156)

## g. POS-Tagging dan Stopword

In [173]:
import stanza
from collections import Counter

In [174]:
# Mengunduh model Bahasa Indonesia
stanza.download('id')
nlp = stanza.Pipeline(lang='id', processors='tokenize,pos')

2025-06-17 20:26:29 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2025-06-17 20:26:29 INFO: Downloading default packages for language: id (Indonesian) ...
2025-06-17 20:26:31 INFO: File exists: C:\Users\HP\stanza_resources\id\default.zip
2025-06-17 20:26:38 INFO: Finished downloading models and saved to C:\Users\HP\stanza_resources
2025-06-17 20:26:38 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-06-17 20:26:40 INFO: Downloaded file to C:\Users\HP\stanza_resources\resources.json
2025-06-17 20:26:40 WARNING: Language id package default expects mwt, which has been added
2025-06-17 20:26:41 INFO: Loading these models for language: id (Indonesian):
| Processor | Package    |
--------------------------
| tokenize  | gsd        |
| mwt       | gsd        |
| pos       | gsd_charlm |

2025-06-17 20:26:41 INFO: Using dev

In [175]:
# Load bobot POS
bobot_pos = pd.read_csv('bobot_pos_tag.csv')

# Konversi bobot ke dictionary
bobot_POS = dict(zip(bobot_pos['POS_Tag'], bobot_pos['Bobot']))

#Load stopword bobot
bobot_stopword = pd.read_csv('bobot_Stopword.csv')

bobot_SW = dict(zip(bobot_stopword['Stopword'], bobot_stopword['Bobot']))

In [176]:
def hitung_skor_pos(teks, bobot_POS):
    doc = nlp(teks)
    tags = [word.upos for sent in doc.sentences for word in sent.words]
    tag_counter = Counter(tags)
    
    total_bobot = 0
    total_tag = 0

    for tag, jumlah in tag_counter.items():
        bobot = bobot_POS.get(tag, 0.5)  # bobot 0.5 jika tag tidak ditemukan
        total_bobot += bobot * jumlah
        total_tag += jumlah

    skor_rata2 = total_bobot / total_tag if total_tag > 0 else 0
    return skor_rata2

# Hitung skor untuk setiap teks
data['rasio_pos_tagging'] = data['teks'].apply(lambda x: hitung_skor_pos(x, bobot_POS))

In [177]:
data.to_csv("posT.csv", index=False)

In [178]:
def hitung_skor_stopword(teks, bobot_SW, stop_words):
    tokens = word_tokenize(teks.lower())
    stopword_tokens = [t for t in tokens if t in stop_words]

    token_counter = Counter(stopword_tokens)
    
    total_bobot = 0
    total_token = 0

    for token, jumlah in token_counter.items():
        bobot = bobot_SW.get(token, 0.5)  # Jika token tidak punya bobot, dianggap 0.5
        total_bobot += bobot * jumlah
        total_token += jumlah

    # Skor rata-rata (normalisasi), bisa juga pakai total_bobot langsung
    skor_rata2 = total_bobot / total_token if total_token > 0 else 0
    return skor_rata2

data['rasio_stopword'] = data['teks'].apply(lambda x: hitung_skor_stopword(x, bobot_SW, stop_words))

In [179]:
data.to_csv('teks_cleaning.csv', index=False)

In [180]:
jumlah_label = data['label'].value_counts()
jumlah_label

label
0    391
1    375
Name: count, dtype: int64

In [181]:
data

,teks,label,panjang_teks,is_noise,is_outlier,teks_normal,teks_stopword,teks_tokenisasi,rasio_panjang_teks,rasio_pos_tagging,rasio_stopword
4,Saya adalah asisten berbasis kecerdasan buatan...,1,435,False,False,saya adalah asisten berbasis kecerdasan buatan...,asisten berbasis kecerdasan buatan dikembangka...,"[asisten, berbasis, kecerdasan, buatan, dikemb...",1,0.606667,0.398998
7,"Untuk membedakan antara **kata benda**, **kata...",1,363,False,False,untuk membedakan antara kata benda kata kerja ...,membedakan benda kerja sifat fungsi peran kali...,"[membedakan, benda, kerja, sifat, fungsi, pera...",1,0.758450,0.241968
8,Jika kita menjatuhkan batu dan kertas dari ket...,1,428,False,False,jika kita menjatuhkan batu dan kertas dari ket...,menjatuhkan batu kertas ketinggian ruangan ham...,"[menjatuhkan, batu, kertas, ketinggian, ruanga...",1,0.608409,0.359802
13,Salah satu contoh kerajinan tradisional dari I...,1,522,False,False,salah satu contoh kerajinan tradisional dari i...,salah contoh kerajinan tradisional indonesia t...,"[salah, contoh, kerajinan, tradisional, indone...",1,0.629012,0.306736
18,"Menurut saya, kebahagiaan adalah perasaan dama...",1,443,False,False,menurut saya kebahagiaan adalah perasaan damai...,kebahagiaan perasaan damai puas pencapaian hal...,"[kebahagiaan, perasaan, damai, puas, pencapaia...",1,0.551927,0.310078
...,...,...,...,...,...,...,...,...,...,...,...
851,Cross-validation adalah salah satu metode pemi...,0,164,False,False,crossvalidation adalah salah satu metode pemis...,crossvalidation salah metode pemisahan data me...,"[crossvalidation, salah, metode, pemisahan, da...",0,0.574987,0.299704
852,Diperlukan utilitas untuk dapat memindahkan at...,0,168,False,False,diperlukan utilitas untuk dapat memindahkan at...,utilitas memindahkan model dilatih komputer jo...,"[utilitas, memindahkan, model, dilatih, komput...",0,0.588953,0.355489
856,Kebebasan pers di Indonesia tidak secara langs...,0,546,False,False,kebebasan pers di indonesia tidak secara langs...,kebebasan pers indonesia langsung pasalpasal u...,"[kebebasan, pers, indonesia, langsung, pasalpa...",1,0.572114,0.269088
859,Tujuan utama dari tugas berbasis klasifikasi a...,0,296,False,False,tujuan utama dari tugas berbasis klasifikasi a...,tujuan utama tugas berbasis klasifikasi mempre...,"[tujuan, utama, tugas, berbasis, klasifikasi, ...",1,0.562554,0.327532
